# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [76]:
%load_ext dotenv
%dotenv 


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [77]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [78]:
import os
from glob import glob


PRICE_DATA = os.getenv("PRICE_DATA")
print(os.getenv("PRICE_DATA"))

parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive = True)
print(parquet_files)
dd_px = dd.read_parquet(parquet_files).set_index("ticker")

print(dd_px.columns)


../../05_src/data/prices/


['../../05_src/data/prices/BKTI/BKTI_2012/part.0.parquet', '../../05_src/data/prices/BKTI/BKTI_2012/part.1.parquet', '../../05_src/data/prices/BKTI/BKTI_2015/part.0.parquet', '../../05_src/data/prices/BKTI/BKTI_2015/part.1.parquet', '../../05_src/data/prices/BKTI/BKTI_2014/part.0.parquet', '../../05_src/data/prices/BKTI/BKTI_2014/part.1.parquet', '../../05_src/data/prices/BKTI/BKTI_2013/part.0.parquet', '../../05_src/data/prices/BKTI/BKTI_2013/part.1.parquet', '../../05_src/data/prices/BKTI/BKTI_1980/part.0.parquet', '../../05_src/data/prices/BKTI/BKTI_1980/part.1.parquet', '../../05_src/data/prices/BKTI/BKTI_1987/part.0.parquet', '../../05_src/data/prices/BKTI/BKTI_1987/part.1.parquet', '../../05_src/data/prices/BKTI/BKTI_1989/part.0.parquet', '../../05_src/data/prices/BKTI/BKTI_1989/part.1.parquet', '../../05_src/data/prices/BKTI/BKTI_1988/part.0.parquet', '../../05_src/data/prices/BKTI/BKTI_1988/part.1.parquet', '../../05_src/data/prices/BKTI/BKTI_1986/part.0.parquet', '../../05_src

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [79]:
dd_shift = dd_px.groupby('ticker', group_keys=False).apply(
    lambda group: group.sort_index().assign(
        Close_lag_1 = group['Close'].shift(1),
        Adj_Close_lag_1 = group['Adj Close'].shift(1)
        )
)

dd_feat = dd_shift.assign(
    Returns = lambda x: x['Close']/x['Close_lag_1'] - 1,
    hi_lo_range = lambda x: x['High'] - x["Low"]
)

dd_feat.compute()

/var/folders/d8/rszv4bcs5rj4g65czmgs5r2r0000gn/T/ipykernel_8014/2068006895.py:1: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_shift = dd_px.groupby('ticker', group_keys=False).apply(


,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,Returns,hi_lo_range
ticker,,,,,,,,,,,,,
A,1999-11-18,32.546494,35.765381,28.612303,31.473534,27.068665,62546300.0,A.csv,1999,NaN,NaN,NaN,7.153078
A,1999-11-19,30.713520,30.758226,28.478184,28.880543,24.838577,15234100.0,A.csv,1999,31.473534,27.068665,-0.082386,2.280043
A,1999-11-22,29.551144,31.473534,28.657009,31.473534,27.068665,6577800.0,A.csv,1999,28.880543,24.838577,0.089783,2.816525
A,1999-11-23,30.400572,31.205294,28.612303,28.612303,24.607880,5975600.0,A.csv,1999,31.473534,27.068665,-0.090909,2.592991
A,1999-11-24,28.701717,29.998211,28.612303,29.372318,25.261524,4843200.0,A.csv,1999,28.612303,24.607880,0.026563,1.385908
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEUS,2003-12-24,6.950000,7.000000,6.880000,6.990000,6.401954,10700.0,ZEUS.csv,2003,6.870000,6.292048,0.017467,0.120000
ZEUS,2003-12-26,7.000000,7.750000,6.800000,7.630000,6.988110,179300.0,ZEUS.csv,2003,6.990000,6.401954,0.091559,0.950000
ZEUS,2003-12-29,7.700000,8.500000,7.670000,8.200000,7.510161,221700.0,ZEUS.csv,2003,7.630000,6.988110,0.074705,0.830000


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [80]:
DF_feat = dd_feat.compute()

DF_feat['Moving_av_returns_10'] = DF_feat['Returns'].rolling(10).mean()



In [ ]:
DF_feat

# If we are working with small dataset, which does not require much memory - it is better to convert our Dask dataframe into pandas dataframe
# for easier calculations. Otherwise, it's better to use Dask as it works better with huge datasets and still can perform the same calculations.
# However, it's not necessary to convert Dask to pandas, it's just more convenient way in our case. 

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,Returns,hi_lo_range,Moving_av_returns_10
ticker,,,,,,,,,,,,,,
A,1999-11-18,32.546494,35.765381,28.612303,31.473534,27.068665,62546300.0,A.csv,1999,NaN,NaN,NaN,7.153078,NaN
A,1999-11-19,30.713520,30.758226,28.478184,28.880543,24.838577,15234100.0,A.csv,1999,31.473534,27.068665,-0.082386,2.280043,NaN
A,1999-11-22,29.551144,31.473534,28.657009,31.473534,27.068665,6577800.0,A.csv,1999,28.880543,24.838577,0.089783,2.816525,NaN
A,1999-11-23,30.400572,31.205294,28.612303,28.612303,24.607880,5975600.0,A.csv,1999,31.473534,27.068665,-0.090909,2.592991,NaN
A,1999-11-24,28.701717,29.998211,28.612303,29.372318,25.261524,4843200.0,A.csv,1999,28.612303,24.607880,0.026563,1.385908,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEUS,2003-12-24,6.950000,7.000000,6.880000,6.990000,6.401954,10700.0,ZEUS.csv,2003,6.870000,6.292048,0.017467,0.120000,0.014296
ZEUS,2003-12-26,7.000000,7.750000,6.800000,7.630000,6.988110,179300.0,ZEUS.csv,2003,6.990000,6.401954,0.091559,0.950000,0.023944
ZEUS,2003-12-29,7.700000,8.500000,7.670000,8.200000,7.510161,221700.0,ZEUS.csv,2003,7.630000,6.988110,0.074705,0.830000,0.028609


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.